<a href="https://colab.research.google.com/github/cars1015/recsys_MyReserch/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Googleドライブをマウント
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ライブラリのインポート

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
!pip install bottleneck
import os
import bottleneck as bn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


データの選択、学習した予測値の読み込み

In [3]:
#データの選択
#data="netflix-prize"
data="ml-20m"
#data="msd"

dir = "/content/drive/MyDrive/recommend/" + data + "/pro_sg/"
pred=np.loadtxt(dir+"pred.txt")

In [5]:
print(pred[:,0:2])

[[ 0.09275124  0.1858637 ]
 [-0.04268303  0.05836733]
 [ 0.00543846 -0.02914956]
 ...
 [ 0.15176699  0.39382971]
 [ 0.04880058  0.01455248]
 [-0.00339534  0.02360744]]


NDCG、Recallの関数

In [7]:
def NDCG(x_pred, x_test, k=100):
  
    user_num = x_pred.shape[0]
    idx_topk_part = bn.argpartition(-x_pred, k, axis=1)
    #ユーザ数と同じ行数の列ベクトルの作成とid_topk_partより最初のk列を取り出している
    topk_part = x_pred[np.arange(user_num)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    #これにより各行を降順にソート
    idx_part = np.argsort(-topk_part, axis=1)
    #ここに各ユーザ上位k個のアイテムのインデックスを格納
    idx_topk = idx_topk_part[np.arange(user_num)[:, np.newaxis], idx_part]
    print("idx_topk")
    print(idx_topk)
    print(idx_topk.shape)
    # ランキングごとの分母部分
    tp = 1. / np.log2(np.arange(2, k + 2))
    print(tp)
    #ユーザごとに上位k個のアイテムに対する評価値を格納
    #x_test[np.arange(user_num)[:, np.newaxis],idx_topk]にはx_testの中に含まれる上位のアイテムのインデックス(idx_topk)を示す                        
    DCG = (x_test[np.arange(user_num)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    print("中身")
    print(x_test[np.arange(user_num)[:,np.newaxis],idx_topk].toarray()*tp)
    #各ユーザに対してk個のアイテムを推奨した場合の理想的な最大のDCGスコアを表す
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in x_test.getnnz(axis=1)])
    #getnnz(axis=1)は各行の非0要素の数を返す。
    return DCG / IDCG

def Recall(x_pred, x_test, k):
    users_num = x_pred.shape[0]

    idx = bn.argpartition(-x_pred, k, axis=1)
    #x_predと同じサイズの全False行列作成
    X_pred_binary = np.zeros_like(x_pred, dtype=bool)
    #ここで推薦されたアイテムがあるインデックスをTrueに置き換え
    X_pred_binary[np.arange(users_num)[:, np.newaxis], idx[:, :k]] = True
    #x_testの要素が0でない要素をTrueにした配列
    X_true_binary = (x_test > 0).toarray()
    #同じ位置にTrueがある場合個数を入れる
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te


In [8]:
unique_sid = list()
with open(os.path.join(dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(dir, 'test_tr.csv'),
    os.path.join(dir, 'test_te.csv'))

N_test=test_data_tr.shape[0]
idx_list_test=range(N_test)
test=test_data_te[idx_list_test]
x=test_data_te.toarray()
print(x)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
n=NDCG(pred,test_data_te,k=100)
r20=Recall(pred,test_data_te,k=20)
r50=Recall(pred,test_data_te,k=50)

idx_topk
[[   5   80  144 ...  155  408    8]
 [   3    7   96 ... 1410  598 1956]
 [ 156   80   71 ...  705  691  939]
 ...
 [  12  100  283 ...  898  278   28]
 [ 391 1372   70 ...   98 1031 1312]
 [ 262  367   89 ... 2201  892  138]]
(10000, 100)
[1.         0.63092975 0.5        0.43067656 0.38685281 0.35620719
 0.33333333 0.31546488 0.30103    0.28906483 0.27894295 0.27023815
 0.26264954 0.25595802 0.25       0.24465054 0.23981247 0.23540891
 0.23137821 0.22767025 0.22424382 0.22106473 0.21810429 0.21533828
 0.21274605 0.21030992 0.2080146  0.20584683 0.20379505 0.20184909
 0.2        0.19823986 0.19656163 0.19495902 0.1934264  0.19195872
 0.19055141 0.18920036 0.18790182 0.18665241 0.18544902 0.18428883
 0.18316925 0.1820879  0.1810426  0.18003133 0.17905223 0.17810359
 0.17718382 0.17629143 0.17542506 0.17458343 0.17376534 0.17296969
 0.17219543 0.1714416  0.17070728 0.16999162 0.16929381 0.1686131
 0.16794878 0.16730018 0.16666667 0.16604765 0.16544255 0.16485086
 0.16427205 0.

In [10]:
print("Test NDCG@100=%.5f" % (np.mean(n)))
print("Test Recall@20=%.5f" % (np.mean(r20)))
print("Test Recall@50=%.5f" % (np.mean(r50)))

Test NDCG@100=0.31519
Test Recall@20=0.27795
Test Recall@50=0.43994
